<a id=0></a>
# Cодержание
[1. Составление корпуса](#1) <br />
[2. Тэггинг](#2)<br />
&emsp;[2.1 PyMorphy](#2.1) <br />
&emsp;[2.2 spaCy](#2.2) <br />
&emsp;[2.3 natasha](#2.3) <br />
[3. Accuracy](#3)<br />
[4. Chunker](#4)<br />
&emsp;[4.1 Паттерны и функция](#4.1) <br />
&emsp;[4.2 Встраивание в дз 1](#4.2) <br />

<a id='1'></a>
# Составление корпуса

Для корпуса я решила отобрать следующие случаи, которые могут быть сложны для парсеров:
- случаи омонимии между словоформами разных частей речи (сущ-глагол: "дуло из окна vs дуло пистолета")
- краткие прилагательные и наречия (иногда даже лингвисту сложно понять что есть что)
- слова типа "что", "как", которые могут быть как союзом, так наречием, так и частицей
- редкие слова, которые могли не встречаться в обучающей выборке и с которыми обычно плохо справляются парсеры
- глаголы с пригласительным наклонением (на -ка) и слова типа непроливайка


Можно еще взять всякие поговорки, потому что они часто построены на омонимии или просто на сопадающих частях слов и часто там встречаются несуществующие слова, которые составлены по каким-нибудь словообразовательным моделям

---
Для работы буду использовать [гугл-таблицу](https://docs.google.com/spreadsheets/d/1DOjjOnxXxhrM4KfY5HIcg11POD-es5Cazf4zRIojO2M/edit?usp=sharing), где и буду хранить предложения и размечать.

In [1]:
import gspread
gc = gspread.service_account(filename='yandex-translator-stories-3ead9c7bb7e0.json')

In [2]:
sh = gc.open("pos_tagger_comparison")
sentences_sheet = sh.worksheet('sentences')

In [3]:
examples = sentences_sheet.col_values(1)
examples

['Я смутно понимал, что значит «отыграюсь», и почему-то решил, что бабушка утопит меня в ванне.',
 'Как Вы говорите, «это очень меня задевает», скажите, пожалуйста что ВЫ подразумеваете под «это»?',
 'Расчесываю волосы на голове, смотрюсь в зеркало, брею себя бритвой,… подбираю галстук к рубашке, понимаю, что мне идёт, а что не идёт, потому что у меня волосы определённого цвета и глаза…',
 'Скажите, а разве не бывает, что и с учителем, имеющим педагогическое образование «мучаются»?',
 'А я же знаю, что учебная часть это решит так: либо 2, либо 3 сейчас',
 '« ― Что ты всё думаешь о себе? ― спрашивает жена.',
 'Да что там подсудимые: на молчание обречены и судьи, и государственный обвинитель, и потерпевшие.',
 'Я видел вашу фотографию. ― Как, обо мне ещё кто-то помнит? Можно, я сейчас приеду?',
 'Как точно мыслил он, как верно угадывал!',
 'Интеллигентный, порядочный, и что мне особо нравится — как они с Олей нашли общий язык.',
 'Книгу читала давно, как только она вышла, и потом перечит

Для начала проверю на сколько словоформ у меня получился мой мини-корпус. Для этого поделю на токены, все слова приведу к нижнему регистру и воспользуюсь каунтером. 

In [4]:
from collections import Counter
from nltk.tokenize import word_tokenize
from string import punctuation

In [5]:
punctuation += '—«»…―'

In [6]:
all_tokens = [t for e in examples for t in word_tokenize(e)]

# Так как я заметила, что там не получилось отлепить многоточие от слова, придется вручную менять
to_change = all_tokens.index('глаза…')
all_tokens[to_change] = 'глаза'
all_tokens.insert(to_change + 1, '…')
#all_tokens

In [7]:
wordforms = Counter([t.lower() for t in all_tokens if t not in punctuation])
print(f'Количество словоформ: {len(wordforms.keys())}')

Количество словоформ: 202


Теперь я запишу все токены в табличку, где буду их размечать. Каждый пример буду отделять пустой строкой

In [8]:
to_write = []
for token in all_tokens:
    if token in punctuation:
        to_write.append([token, 'PUNCT'])
    else:
        to_write.append([token])
# sh.worksheet('tokens').append_rows(to_write)

При разметке я в случае неоднозначных моментов опиралась на разметку нкря (тк большая часть примеров была взята оттуда (я старалась брыть примеры со снятой омонимией)) + на то, какие тэги UD ставятся в [SynTagRus](https://universaldependencies.org/treebanks/ru_syntagrus/index.html)
Спорные моменты, которые я не знала как лучше размечать, я выделила в гугл табличке

<a id='2'></a>
# Тэггинг при помощи библиотек

<a id='2.1'></a>
## pymorphy
Не использует контекст, парсит только пословно. Так как он предлагает несколько вариантов разбора, я буду учитывать тот, который с большим скором.
[(наверх)](#0)

In [9]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Для преобразования одного тегсета в другой буду использовать библиотеку [russian-tagsets](https://pypi.org/project/russian-tagsets/) (зачем изобреть то, что уже есть?)

In [10]:
#!pip install russian-tagsets
from russian_tagsets import converters

In [11]:
oc_to_ud = converters.converter('opencorpora-int', 'ud20')

In [12]:
pymorphy_result = []
for token in all_tokens:
    tags = morph.parse(token)[0].tag
    if tags.POS:
        # буду для конвертера учитывать все теги, потому что, кажется, модуль для разрешения спорных случаев использует 
        # другие грамматические теги
        converted_pos = oc_to_ud(str(tags)).split(' ')[0]
    else:
        converted_pos = 'PUNCT' # так как pymorphy не считает pnct тегом 
    pymorphy_result.append([token, tags.POS, converted_pos])
pymorphy_result

[['Я', 'NPRO', 'PRON'],
 ['смутно', 'ADVB', 'ADV'],
 ['понимал', 'VERB', 'VERB'],
 [',', None, 'PUNCT'],
 ['что', 'CONJ', 'CCONJ'],
 ['значит', 'CONJ', 'CCONJ'],
 ['«', None, 'PUNCT'],
 ['отыграюсь', 'VERB', 'VERB'],
 ['»', None, 'PUNCT'],
 [',', None, 'PUNCT'],
 ['и', 'CONJ', 'CCONJ'],
 ['почему-то', 'ADVB', 'ADV'],
 ['решил', 'VERB', 'VERB'],
 [',', None, 'PUNCT'],
 ['что', 'CONJ', 'CCONJ'],
 ['бабушка', 'NOUN', 'NOUN'],
 ['утопит', 'VERB', 'VERB'],
 ['меня', 'NPRO', 'PRON'],
 ['в', 'PREP', 'ADP'],
 ['ванне', 'NOUN', 'NOUN'],
 ['.', None, 'PUNCT'],
 ['Как', 'CONJ', 'CCONJ'],
 ['Вы', 'NPRO', 'PRON'],
 ['говорите', 'VERB', 'VERB'],
 [',', None, 'PUNCT'],
 ['«', None, 'PUNCT'],
 ['это', 'PRCL', 'PART'],
 ['очень', 'ADVB', 'ADV'],
 ['меня', 'NPRO', 'PRON'],
 ['задевает', 'VERB', 'VERB'],
 ['»', None, 'PUNCT'],
 [',', None, 'PUNCT'],
 ['скажите', 'VERB', 'VERB'],
 [',', None, 'PUNCT'],
 ['пожалуйста', 'PRCL', 'PART'],
 ['что', 'CONJ', 'CCONJ'],
 ['ВЫ', 'NPRO', 'PRON'],
 ['подразумеваете',

In [13]:
oc_pymorphy = ['oc_pymorphy']
oc_pymorphy.extend([p[1] for p in pymorphy_result])
ud_pymorphy = ['ud_pymorphy']
ud_pymorphy.extend([p[2] for p in pymorphy_result])
#sh.worksheet('tokens').insert_cols([oc_pymorphy, ud_pymorphy], col=3)

<a id='2.2'></a>
## spacy
Парсит по предложениям, должен учитывать контекст
[(наверх)](#0)

In [14]:
!pip install -U spacy>=3.0

In [15]:
import spacy
!python -m spacy download ru_core_news_md

Defaulting to user installation because normal site-packages is not writeable
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [16]:
nlp = spacy.load('ru_core_news_md')

Так как spacy указывает теги в формате UD, который я и использую, то ничего менять не надо

In [17]:
spacy_results = []
for ex in examples:
    processed_ex = nlp(ex)
    for t in processed_ex:
        spacy_results.append([t.text, t.pos_])
spacy_results

[['Я', 'PRON'],
 ['смутно', 'ADV'],
 ['понимал', 'VERB'],
 [',', 'PUNCT'],
 ['что', 'SCONJ'],
 ['значит', 'VERB'],
 ['«', 'PUNCT'],
 ['отыграюсь', 'VERB'],
 ['»', 'PUNCT'],
 [',', 'PUNCT'],
 ['и', 'CCONJ'],
 ['почему', 'ADV'],
 ['-', 'ADV'],
 ['то', 'ADV'],
 ['решил', 'VERB'],
 [',', 'PUNCT'],
 ['что', 'SCONJ'],
 ['бабушка', 'NOUN'],
 ['утопит', 'VERB'],
 ['меня', 'PRON'],
 ['в', 'ADP'],
 ['ванне', 'NOUN'],
 ['.', 'PUNCT'],
 ['Как', 'SCONJ'],
 ['Вы', 'PRON'],
 ['говорите', 'VERB'],
 [',', 'PUNCT'],
 ['«', 'PUNCT'],
 ['это', 'PRON'],
 ['очень', 'ADV'],
 ['меня', 'PRON'],
 ['задевает', 'VERB'],
 ['»', 'PUNCT'],
 [',', 'PUNCT'],
 ['скажите', 'VERB'],
 [',', 'PUNCT'],
 ['пожалуйста', 'ADV'],
 ['что', 'SCONJ'],
 ['ВЫ', 'PROPN'],
 ['подразумеваете', 'VERB'],
 ['под', 'ADP'],
 ['«', 'PUNCT'],
 ['это', 'PRON'],
 ['»', 'PUNCT'],
 ['?', 'PUNCT'],
 ['Расчесываю', 'VERB'],
 ['волосы', 'NOUN'],
 ['на', 'ADP'],
 ['голове', 'NOUN'],
 [',', 'PUNCT'],
 ['смотрюсь', 'VERB'],
 ['в', 'ADP'],
 ['зеркало', 

Но из-за того, что spacy сам токенизирует предложения, надо проверить, что у нас одинаково разбилось на токены, чтобы в табличке ничего не поползло. Для начала проверим по длине

In [18]:
spacy_tokens = [r[0] for r in spacy_results]
len(spacy_tokens), len(all_tokens)

(370, 364)

Длины разные, значит, что-то разбивается по-разному. Посмотрим на различия:

In [19]:
set(spacy_tokens) ^ set(all_tokens)

{'-', 'кто', 'кто-то', 'по-прежнему', 'почему', 'почему-то', 'прежнему', 'то'}

Видно, что токенизатор spacy сплитит слова с дефисом. Попробую убрать эту функцию и пропарсить еще раз. 
Я нашла на [cайте](https://spacy.io/usage/linguistic-features#tokenization) код, который должен убрать из токенизатора правило разбивать по дефису

In [20]:
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [21]:
spacy_results = []
for ex in examples:
    processed_ex = nlp(ex)
    for t in processed_ex:
        spacy_results.append([t.text, t.pos_])

In [22]:
spacy_tokens = [r[0] for r in spacy_results]
assert spacy_tokens == all_tokens

Теперь все хорошо, можно записывать в гугл табличку:

In [23]:
ud_spacy = ['ud_spacy']
ud_spacy.extend([p[1] for p in spacy_results])
#sh.worksheet('tokens').insert_cols([ud_spacy], col=5)

<a id='2.3'></a>
## natasha

Наташа тоже использует теги UD [(наверх)](#0)

In [24]:
pip install natasha

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [26]:
natasha_results = []
for ex in examples:
    doc = Doc(ex)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for t in doc.tokens:
        natasha_results.append([t.text, t.pos])
natasha_results

[['Я', 'PRON'],
 ['смутно', 'ADV'],
 ['понимал', 'VERB'],
 [',', 'PUNCT'],
 ['что', 'SCONJ'],
 ['значит', 'VERB'],
 ['«', 'PUNCT'],
 ['отыграюсь', 'NOUN'],
 ['»', 'PUNCT'],
 [',', 'PUNCT'],
 ['и', 'CCONJ'],
 ['почему-то', 'ADV'],
 ['решил', 'VERB'],
 [',', 'PUNCT'],
 ['что', 'SCONJ'],
 ['бабушка', 'NOUN'],
 ['утопит', 'VERB'],
 ['меня', 'PRON'],
 ['в', 'ADP'],
 ['ванне', 'NOUN'],
 ['.', 'PUNCT'],
 ['Как', 'ADV'],
 ['Вы', 'PRON'],
 ['говорите', 'VERB'],
 [',', 'PUNCT'],
 ['«', 'PUNCT'],
 ['это', 'PRON'],
 ['очень', 'ADV'],
 ['меня', 'PRON'],
 ['задевает', 'VERB'],
 ['»', 'PUNCT'],
 [',', 'PUNCT'],
 ['скажите', 'VERB'],
 [',', 'PUNCT'],
 ['пожалуйста', 'ADV'],
 ['что', 'SCONJ'],
 ['ВЫ', 'PROPN'],
 ['подразумеваете', 'VERB'],
 ['под', 'ADP'],
 ['«', 'PUNCT'],
 ['это', 'PRON'],
 ['»', 'PUNCT'],
 ['?', 'PUNCT'],
 ['Расчесываю', 'ADJ'],
 ['волосы', 'NOUN'],
 ['на', 'ADP'],
 ['голове', 'NOUN'],
 [',', 'PUNCT'],
 ['смотрюсь', 'VERB'],
 ['в', 'ADP'],
 ['зеркало', 'NOUN'],
 [',', 'PUNCT'],
 ['бр

In [27]:
natasha_tokens = [res[0] for res in natasha_results]
len(natasha_tokens), len(all_tokens)

(365, 364)

Опять что-то по-разному токенизировалось

In [28]:
set(natasha_tokens) ^ set(all_tokens)

{'a', 'перепел', 'перепелa'}

Почему-то natasha поделила "перепела" на два токена... Но так как это не повлияло на разбор(я попробовала ниже исправить токены и сделать заново разбор), я просто из итогового списка удалю лишний токен. 

In [29]:
ex = examples[-1]
doc = Doc(ex)
doc.segment(segmenter)
doc.tokens[1].text = 'перепела'
doc.tokens[1].stop = doc.tokens[2].stop
del doc.tokens[2]

doc.tag_morph(morph_tagger)
doc.tokens

[DocToken(stop=1, text='У', pos='ADP'),
 DocToken(start=2, stop=10, text='перепела', pos='NOUN', feats=<Anim,Gen,Fem,Sing>),
 DocToken(start=11, stop=12, text='и', pos='CCONJ'),
 DocToken(start=13, stop=22, text='перепелки', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=23, stop=27, text='пять', pos='NUM', feats=<Acc>),
 DocToken(start=28, stop=37, text='перепелят', pos='NOUN', feats=<Anim,Gen,Masc,Plur>),
 DocToken(start=37, stop=38, text='.', pos='PUNCT')]

In [30]:
del natasha_results[natasha_tokens.index('a')]
natasha_tokens = [res[0] for res in natasha_results]
assert spacy_tokens == all_tokens

Теперь можно записать в таблицу

In [31]:
ud_natasha = ['ud_natasha']
ud_natasha.extend([p[1] for p in natasha_results])
#sh.worksheet('tokens').insert_cols([ud_natasha], col=6)

<a id=3></a>
# Считаем accuracy 
[(наверх)](#0)

In [32]:
my_pos_tags = sh.worksheet('tokens').col_values(2)[1:]
my_pos_tags

['PRON',
 'ADV',
 'VERB',
 'PUNCT',
 'PRON',
 'VERB',
 'PUNCT',
 'VERB',
 'PUNCT',
 'PUNCT',
 'CCONJ',
 'ADV',
 'VERB',
 'PUNCT',
 'SCONJ',
 'NOUN',
 'VERB',
 'PRON',
 'ADP',
 'NOUN',
 'PUNCT',
 'SCONJ',
 'PRON',
 'VERB',
 'PUNCT',
 'PUNCT',
 'PRON',
 'ADV',
 'PRON',
 'VERB',
 'PUNCT',
 'PUNCT',
 'VERB',
 'PUNCT',
 'PART',
 'PRON',
 'PRON',
 'VERB',
 'ADP',
 'PUNCT',
 'PRON',
 'PUNCT',
 'PUNCT',
 'VEBR',
 'NOUN',
 'ADP',
 'NOUN',
 'PUNCT',
 'VERB',
 'ADP',
 'NOUN',
 'PUNCT',
 'VERB',
 'PRON',
 'NOUN',
 'PUNCT',
 'PUNCT',
 'VERB',
 'NOUN',
 'ADP',
 'NOUN',
 'PUNCT',
 'VERB',
 'PUNCT',
 'PRON',
 'PRON',
 'VERB',
 'PUNCT',
 'CCONJ',
 'PRON',
 'PART',
 'VERB',
 'PUNCT',
 'ADV',
 'SCONJ',
 'ADP',
 'PRON',
 'NOUN',
 'ADJ',
 'NOUN',
 'CCONJ',
 'NOUN',
 'PUNCT',
 'VERB',
 'PUNCT',
 'CCONJ',
 'PART',
 'PART',
 'VERB',
 'PUNCT',
 'SCONJ',
 'CCONJ',
 'ADP',
 'NOUN',
 'PUNCT',
 'VERB',
 'ADJ',
 'NOUN',
 'PUNCT',
 'VERB',
 'PUNCT',
 'PUNCT',
 'CCONJ',
 'PRON',
 'PART',
 'VERB',
 'PUNCT',
 'SCONJ',


In [33]:
from sklearn.metrics import accuracy_score

In [34]:
for i in range(4, 7):
    tags = sh.worksheet('tokens').col_values(i)
    acc = accuracy_score(my_pos_tags, tags[1:])
    tagger = tags[0].split('_')[1]
    print(f'Accuracy for {tagger}: {acc}')

Accuracy for pymorphy: 0.8406593406593407
Accuracy for spacy: 0.9010989010989011
Accuracy for natasha: 0.8846153846153846


<a id=4></a>
# Chunker
[(наверх)](#0)

<a id=4.1></a>
## Паттерны и функция

Паттерны, которые я решила выбрать: 
1. не + составное глаг сказ (чтобы сюда попали сочетания типа "*__не стану__ еще раз __брать__", "Я __не могу посоветовать__ эту сыворотку*")
2. не + глаг ("*К покупке __не рекомендую__!", "Крем я __не__ особо __советую__*") 
3. не + глаг + сущ (всякие "*__Не сушит__ кожу", "__не вызывает__ аллергию")
4. не + прилаг (так как говорит о том, какого качества у продукта нет - важно для негативных отзывов)
5. наречие + глагол, глагол + наречие (хорошо распределяется и т.п., когда сам глагол не имеет полярности, а только в сочетании с наречием)

In [35]:
import re

In [36]:
def clean(text):
    text = text.replace('\xa0', ' ')
    text = re.sub(r'[a-zA-Z]+', '', text)
    return text

In [37]:
def tagger(text):
    text_pos = []
    doc = nlp(text)
    for sent in doc.sents:
        tagged_sent = ''
        for t in sent:
            tagged_sent += t.lemma_ + '_' + t.pos_ + ' '
        text_pos.append(tagged_sent)
    return text_pos

In [38]:
def token_pattern_builder(lemma='[А-я]*?', tag='\w*'):
    return r'({lemma}_({tag}) )'.format(lemma=lemma, tag=tag)

In [39]:
def chunker(clean_text):
    
    verb = token_pattern_builder(tag='VERB')
    no = token_pattern_builder(lemma='не')
    verb_aux = token_pattern_builder(tag='VERB|AUX')
    adj = token_pattern_builder(tag='ADJ')
    modif = token_pattern_builder(tag='ADV')
    noun = token_pattern_builder(tag='NOUN')
    tags = 'ADV|NOUN|PRON|ADJ|DET'
    extra = token_pattern_builder(tag=tags)
    
    patterns = [re.compile(no + verb_aux + extra + '*?' + verb, re.IGNORECASE), # не + составное глаг сказ
                re.compile(no + modif + '*?' + adj, re.IGNORECASE), # не + прилаг
                re.compile(modif + verb, re.IGNORECASE), # наречие + глагол
                re.compile(verb + modif, re.IGNORECASE), # глаг + наречие
                re.compile(no + modif + '*?' + verb, re.IGNORECASE), # не + глаг
                re.compile(no + verb + modif + '*?' + noun, re.IGNORECASE), # не + перех глагол
               ]
    chunk_elements = [
        (0, 2, -2),
        (0, -2), 
        (0, 2), 
        (0, 2),
        (0, -2),
        (0, 2, -2)
    ]

    chunks = []
    for sent in tagger(clean_text):
        for pattern, indexes in zip(patterns, chunk_elements):
            matches = re.findall(pattern, sent)
            if matches:
                for match in matches:
                    chunk_lemmas = [match[i].split('_')[0].strip() for i in indexes]
                    chunks.append(' '.join(chunk_lemmas))
    return chunks

<a id=4.2></a>
## Встраивание в дз1
[наверх](#0)

In [40]:
with open('positive.txt', 'r', encoding='utf-8-sig')as f:
    positive_reviews = f.readlines()
with open('negative.txt', 'r', encoding='utf-8-sig')as f:
    negative_reviews = f.readlines()

In [41]:
# разбиваем на выборки
pos_train, pos_test = positive_reviews[:30], positive_reviews[30:]
neg_train, neg_test = negative_reviews[:30], negative_reviews[30:]

In [42]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')
sw = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kra$0t04ka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Так как алгоритм в прошлой домашке заключался в составлении словаря для позитивных и негативных отзывов, то, чтобы встроить чанкер, надо будет к обычным леммам добавить чанки.

In [80]:
def chunk_normalizer(text):
    text = clean(text)
    chunks = chunker(text)
    tokenized = nlp(text)
    without_sw = [w.lemma_ for w in tokenized if w.text not in sw]
    norm_tokens = [w for w in without_sw if w not in punctuation]
    norm_tokens = [w for w in norm_tokens if not w.isspace()]
    norm_tokens.extend(chunks)
    return norm_tokens

In [82]:
from nltk.tokenize import TweetTokenizer, sent_tokenize
from pymorphy2 import MorphAnalyzer
def normalizer(text):
    text = text.replace('\xa0', ' ')
    text = re.sub(r'[a-zA-Z]+', '', text)
    tknzr = TweetTokenizer()
    morph = MorphAnalyzer()
    tokenized = tknzr.tokenize(text)
    without_sw = [w for w in tokenized if w not in sw]
    clean_text = [w for w in without_sw if w not in punctuation]
    normalized = [morph.parse(w.lower())[0].normal_form for w in clean_text]
    return normalized

In [44]:
from collections import Counter
import numpy as np
import random

In [85]:
class NaiveSentimnetAnalysis():
    def __init__(self, normalizer=normalizer):
        self.normalizer = normalizer
        
    def train(self, pos_train, neg_train):
        positive_words = [w for rev in pos_train for w in self.normalizer(rev)]
        negative_words = [w for rev in neg_train for w in self.normalizer(rev)]
        positive_vocab = Counter(positive_words)
        negative_vocab = Counter(negative_words)
        only_neg = set(negative_vocab.keys()) - set(positive_vocab.keys())
        only_pos = set(positive_vocab.keys()) - set(negative_vocab.keys())
        self.only_pos_freq = {w: c / len(positive_vocab) * 100 for w, c in positive_vocab.items() if w in only_pos and c > 0}
        self.only_neg_freq = {w: c / len(negative_vocab) * 100 for w, c in negative_vocab.items() if w in only_neg and c > 0}
    
    def predict(self, test):
        predictions = []
        for review in test:
            normalized = self.normalizer(review)
            rate = 0
            for w in normalized:
                if self.only_pos_freq.get(w):
                    rate += self.only_pos_freq[w]
                elif self.only_neg_freq.get(w):
                    rate -= self.only_neg_freq[w]
            predictions.append(np.sign(rate))
        return predictions

In [76]:
test = []
test.extend([(r, 1) for r in pos_test])
test.extend([(r, -1) for r in neg_test])
random.shuffle(test)
y_test = [r[1] for r in test]
x_test = [r[0] for r in test]

In [86]:
regular = NaiveSentimnetAnalysis(normalizer)
regular.train(pos_train, neg_train)
y_pred = regular.predict(x_test)
print(f'Accuracy в прошлой домашке: {accuracy_score(y_pred, y_test)}')

Accuracy в прошлой домашке: 0.7


In [87]:
with_chunks = NaiveSentimnetAnalysis(chunk_normalizer)
with_chunks.train(pos_train, neg_train)
y_pred = with_chunks.predict(x_test)
print(f'Accuracy c чанкером: {accuracy_score(y_pred, y_test)}')

Accuracy c чанкером: 0.7


Ничего не поменялось...🤡